# Data Cleaning

**Input**: Raw data stored in S3 esported from PostgreSQL with .csv format

**Output**: Cleaned data throught AWS Sagemaker - Data Wranglser stored in S3 with .csv format usable from AWS Sagemaker Algorithms.

To make it easier to understand what AWS Sagemaker - Data Wrangler does we are going to repeat the process step by step here with a small amount of data.

The output of this notebook are three different datasets that will be used for training, validation and test. We are going to show the steps used to create *numerical* only datasets.  But also mixed numerical and categorical datasets will be created, in another flow, to use with Sagemaker Algorithms that support categorical data.

# AWS Sagemaker - Data Wrangler

[Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)

AWS SageMaker Data Wrangler is a data preparation and visualization tool provided by Amazon Web Services (AWS) for building machine learning models. It simplifies the process of data preparation by providing a visual interface for cleaning, transforming, and combining data from different sources.

We will use it to analyze raw data and process them into cleaned data for our models.

In the following cells we are going to describe step by step all pre-processing jobs done with the Data Wrangler.

## All Data Wrangler Steps

<img src="../images/all_steps.png" alt="All Data Wrangler Steps" width="auto"/>

As you can see the process is a bit overwhelming.

### Step 1 and 2: Import data source

In the first step we import our data. The actual file imported in the Data Wrangler flow has a 4 GB size. Since we want to show the steps in this note book we are going load a smaller version.

In the Data Wrangler dashboard we can see a "Data type" step that is going to infer data types of the sourced data. We are going to skip this step here in this notebook.

In [2]:
import pandas as pd
data_source_s3_uri = "s3://ds-houseprices/export_all_ads.csv"
df = pd.read_csv(data_source_s3_uri, nrows=10000)
df.shape

(10000, 2)

### Steps 3 and 4: Flatten data column and data_room_info json strings
We are going to flatten *data* and the nested *room_info* json strings and split its values in multiple columns.
For privacy we are going to remove also *data_url* and *data_site*.

In [3]:
import json
from pandas import json_normalize

def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    return json.loads(d)

df.dropna(inplace=True)
data_df = pd.merge(
    df[['building_type']], 
    json_normalize(df['data'].apply(only_dict).tolist(), sep='_').add_prefix('data_'), 
    how="inner", left_index=True, right_index=True)
data_df.shape

(9986, 50)

### Step 5: Drop unused columns

In this step we are going to drop some unused columns, that will not give useful information since their values are all NaN or identical values.

In [4]:
unused_columns = [
            "building_type",
            "data_address",
            "data_agency_number",
            "data_agency_url",
            "data_cadastral_class",
            "data_cadastral_income",
            "data_description",
            "data_doorman",
            "data_download_date",
            "data_facade",
            "data_favorite_id",
            "data_garret",
            "data_is_active",
            "data_note",
            "data_preview_image_url",
            "data_reference_number",
            "data_removed_date",
            "data_room_info_air_conditioning",
            "data_room_info_available_bed",
            "data_room_info_bed_price",
            "data_room_info_building_fees",
            "data_room_info_building_type",
            "data_room_info_contract",
            "data_room_info_other_expenses",
            "data_room_info_private_bath",
            "data_room_info_room_ad_type",
            "data_room_info_room_number",
            "data_room_info_room_people",
            "data_room_info_room_type",
            "data_sold",
            "data_status_code",
            "data_street_price",
            "data_terrace",
            "data_url",
            "data_site"
          ]

data_df.drop(columns=unused_columns, axis=1, inplace=True)
data_df

,data_box,data_size,data_baths,data_floor,data_price,data_rooms,data_title,data_agency,data_arredo,data_coords,data_status,data_balcony,data_elevator,data_publish_date,data_room_info_feature_list
0,1 in garage/box,68.0,1.0,0,115000.0,2.0,"Bilocale via Comotto, Villafranca di Verona",Immobiliare Duomo,,"45.3839,10.9187",Buono / Abitabile,None,unknown,2023-02-22 18:45:20,[]
1,1 in garage/box,80,2.0,3° piano,155000.0,3.0,"Trilocale via Bernina 21, Centro, Camparada",TEMPOCASA CASATENOVO,,"45.6549,9.32357",Buono / Abitabile,None,unknown,2022-09-21 00:00:00,"[Cancello elettrico, Mansarda, Porta blindata,..."
2,1 in garage/box,348,3.0,Piano terra,400000.0,5.0,"Appartamento viale Roma 12, Masarone, Biella",IMMOBILBIELLA,,"45.5563,8.0653",Ottimo / Ristrutturato,si,unknown,2022-04-21 00:00:00,"[Caminetto, Impianto di allarme, Porta blindat..."
3,1 in garage/box,90,1.0,1°,65000.0,4.0,"Quadrilocale Strada della Nera 21, Piazzo, Van...",Immobiliare Aguiari,,"45.5709,8.0406",Ottimo / Ristrutturato,si,si,2022-02-03 00:00:00,"[Porta blindata, Balcone, Impianto tv centrali..."
4,"1 in garage/box, 1",98,1.0,Piano terra,75000.0,4.0,"Quadrilocale Lorazzo Inferiore 10, Andorno Cac...",RE/MAX Unit srl,,"45.6016,8.0525",Buono / Abitabile,None,unknown,2022-06-01 00:00:00,"[Caminetto, Fibra ottica, Armadio a muro, Impi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,,250,2.0,,35000.0,3.0,"Terratetto plurifamiliare via Gilera, Centro, ...",Affiliato Tecnocasa: TECNOTIRANO SRL,,"46.2123,10.1733",Da ristrutturare,si,unknown,2022-05-19 00:00:00,"[Balcone, Parzialmente Arredato, Cantina, Giar..."
9989,1 in garage/box,45,1.0,1 piano: Piano terra,110000.0,2.0,"Bilocale via Galet 1, Valdisotto",RE/MAX Next,,"46.4624,10.3415",Ottimo / Ristrutturato,si,si,2022-07-08 00:00:00,"[Cancello elettrico, Fibra ottica, Porta blind..."
9990,2 in garage/box,108,2.0,1 piano: Piano rialzato,\n € 289.000\n ...,4.0,"Villa bifamiliare Località Graile 20, Sondalo",Privato,,"46.34329987,10.35447979",Ottimo / Ristrutturato,si,unknown,2022-09-02 00:00:00,"[Cucina, 1 Balcone, Arredato, Giardino privato..."
9991,"2 in garage/box, 8",450,3.0,2 piani: Piano terra,\n € 420.000\n ...,5.0,"Villa plurifamiliare, da ristrutturare, 450 m²...",Agenzia Immobiliare G.Esse Case sas,,"46.2451,10.4702",Da ristrutturare,si,unknown,2022-10-11 00:00:00,"[Caminetto, Mansarda, Esposizione esterna, Bal..."


### Steps from 6 to 11: cast column types
We are going cast type of the following columns.

In [5]:
cast_df = data_df.copy()
cast_df.dtypes

data_box                        object
data_size                       object
data_baths                     float64
data_floor                      object
data_price                      object
data_rooms                     float64
data_title                      object
data_agency                     object
data_arredo                     object
data_coords                     object
data_status                     object
data_balcony                    object
data_elevator                   object
data_publish_date               object
data_room_info_feature_list     object
dtype: object

In [6]:
# Step 6: data_room_info_feature_list to string
cast_df["data_room_info_feature_list"] = cast_df["data_room_info_feature_list"].astype("string")

# Step 7: data_price to float
cast_df["data_price"] = pd.to_numeric(cast_df["data_price"], errors="coerce") # <- Note that here possible NaN are a result. Some data_prices contain € or multiple values ex. 100.000 € - 150.000 € 

# Step 8: data_size to float
cast_df["data_size"] = pd.to_numeric(cast_df["data_size"], errors="coerce")

# Step 9: data_rooms to long
cast_df["data_rooms"] = cast_df["data_rooms"].astype('Int64')

# Step 10: data_baths to long
cast_df["data_baths"] = cast_df["data_baths"].astype('Int64')

# Step 11: data_publish_date to datetime
cast_df["data_publish_date"] = pd.to_datetime(cast_df["data_publish_date"]) # <- format is yyyy-mm-dd hh:MM:ss ex. 2022-09-21 00:00:00

cast_df.dtypes

data_box                               object
data_size                             float64
data_baths                              Int64
data_floor                             object
data_price                            float64
data_rooms                              Int64
data_title                             object
data_agency                            object
data_arredo                            object
data_coords                            object
data_status                            object
data_balcony                           object
data_elevator                          object
data_publish_date              datetime64[ns]
data_room_info_feature_list            string
dtype: object

### Step 12: Extract columns from data_publish_date
In this step we are going to split the publish date in three new columns for year, quarter and month.

In [7]:
date_df = cast_df.copy()

# Create new columns
date_df['data_publish_date_year'] = date_df['data_publish_date'].dt.year.astype('Int64')
date_df['data_publish_date_quarter'] = date_df['data_publish_date'].dt.quarter.astype('Int64') -1 
date_df['data_publish_date_month'] = date_df['data_publish_date'].dt.month.astype('Int64') -1 
date_df [["data_publish_date","data_publish_date_year", "data_publish_date_quarter", "data_publish_date_month"]]

,data_publish_date,data_publish_date_year,data_publish_date_quarter,data_publish_date_month
0,2023-02-22 18:45:20,2023,0,1
1,2022-09-21 00:00:00,2022,2,8
2,2022-04-21 00:00:00,2022,1,3
3,2022-02-03 00:00:00,2022,0,1
4,2022-06-01 00:00:00,2022,1,5
...,...,...,...,...
9988,2022-05-19 00:00:00,2022,1,4
9989,2022-07-08 00:00:00,2022,2,6
9990,2022-09-02 00:00:00,2022,2,8
9991,2022-10-11 00:00:00,2022,3,9


### Step 13: Robust standard deviation numeric outliers

Considering **data_price** and **data_size** columns we are going to apply a trasformation to identify outliers.

 
From AWS Sagemaker Data Wrangler documentation: 

https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html#data-wrangler-transform-handle-outlier

- The Data Wrangler transformation detect and fix outliers in numeric features using statistics that are robust to outliers.
- You must define an Upper quantile and a Lower quantile for the statistics used to calculate outliers.
- You must also specify the number of Standard deviations from which a value must vary from the mean to be considered an outlier.

(For example, if you specify 3 for Standard deviations, a value must fall more than 3 standard deviations from the mean to be considered an outlier.)

*The Fix method is the method used to handle outliers when they are detected. You can choose from the following:*

- ***Clip**: Use this option to clip the outliers to the corresponding outlier detection bound.*
- ***Remove**: Use this option to remove rows with outliers from the dataframe.*
- ***Invalidate**: Use this option to replace outliers with invalid values.*

We are going to use 0.99 as upper quantile and 0.01 as lower quantile and 4 standard deviation. And apply this process to remove outliers in data_price and data_size columns.

<img src="../images/13_remove_outliers.png" alt="13. Remove outlier with Data Wrangler Robust standard deviation" width="auto"/>

In [8]:
import numpy as np

outlier_df = date_df.copy()
print(outlier_df.shape)
print()

outlier_df['to_remove'] = False
for column in ["data_price", "data_size"]:

    print(f"COLUMN: {column}")

    upper_quantile = outlier_df[column].quantile(q=0.99)
    lower_quantile = outlier_df[column].quantile(q=0.01)
    robust_df = outlier_df[outlier_df[column].between(lower_quantile, upper_quantile)]
    
    print(f"UPPER QUANTILE: {upper_quantile}")
    print(f"LOWER QUANTILE: {lower_quantile}")
    
    robust_mean = robust_df[column].mean()
    robust_std =  robust_df[column].std()
    
    print(f"ROBUST MEAN: {robust_mean}")
    print(f"ROBUST STANDARD DEVIATION: {robust_std }" )
    
    upper_threshold = robust_mean + robust_std * 4
    lower_threshold = max(robust_mean -robust_std * 4, 0)
    
    print(f"UPPER THRESHOLD: {upper_threshold}")
    print(f"LOWER THRESHOLD: {lower_threshold}")
    
    outlier_df['to_remove'] =outlier_df[column].between(lower_threshold, upper_threshold)
    
    print()
    
outlier_df = outlier_df[outlier_df["to_remove"]]
outlier_df.drop("to_remove", axis=1, inplace=True)
outlier_df.shape

(9986, 18)

COLUMN: data_price
UPPER QUANTILE: 2027000.0000000438
LOWER QUANTILE: 23000.0
ROBUST MEAN: 251038.7314864145
ROBUST STANDARD DEVIATION: 247970.3988674684
UPPER THRESHOLD: 1242920.3269562882
LOWER THRESHOLD: 0

COLUMN: data_size
UPPER QUANTILE: 1000.0
LOWER QUANTILE: 35.0
ROBUST MEAN: 150.62559918408974
ROBUST STANDARD DEVIATION: 121.58338135806466
UPPER THRESHOLD: 636.9591246163484
LOWER THRESHOLD: 0



(9761, 18)

### Step 14: Drop missing values
In this step we are going to drop missing values for *data_price*, *data_size*. and *data_coords*.

In [9]:
missing_df = outlier_df.copy()

missing_df = missing_df[missing_df["data_price"].notna()]
missing_df = missing_df[missing_df["data_size"].notna()]
missing_df = missing_df[missing_df["data_coords"].notna()]
missing_df[["data_price","data_size","data_coords"]]

,data_price,data_size,data_coords
0,115000.0,68.0,"45.3839,10.9187"
1,155000.0,80.0,"45.6549,9.32357"
2,400000.0,348.0,"45.5563,8.0653"
3,65000.0,90.0,"45.5709,8.0406"
4,75000.0,98.0,"45.6016,8.0525"
...,...,...,...
9986,60000.0,99.0,"46.1786,10.3291"
9987,92000.0,110.0,"46.2507,10.2361"
9988,35000.0,250.0,"46.2123,10.1733"
9989,110000.0,45.0,"46.4624,10.3415"


### Step 15 and 16: Remove symbols

In this step we are going to transform columns removing symbols.
Additionally, we will replace commas in data_room_info_feature_list with spaces.

In [10]:
remove_symbols_df = missing_df.copy()
columns_with_symbols = [
            "data_agency",
            "data_arredo",
            "data_balcony",
            "data_box",
            "data_elevator",
            "data_floor",
            "data_room_info_feature_list",
            "data_status",
            "data_title"
          ]

for column in columns_with_symbols:
    remove_symbols_df[column] = remove_symbols_df[column].str.replace(r'[/\[\]!@#$%^&*()_+=\\~{}|<>?;\"\'\-]', "", regex=True)
    if column == "data_room_info_feature_list":
        remove_symbols_df[column]  = remove_symbols_df[column].str.replace(",", " ", regex=False)

remove_symbols_df[columns_with_symbols]

,data_agency,data_arredo,data_balcony,data_box,data_elevator,data_floor,data_room_info_feature_list,data_status,data_title
0,Immobiliare Duomo,,None,1 in garagebox,unknown,NaN,,Buono Abitabile,"Bilocale via Comotto, Villafranca di Verona"
1,TEMPOCASA CASATENOVO,,None,1 in garagebox,unknown,3° piano,Cancello elettrico Mansarda Porta blindata ...,Buono Abitabile,"Trilocale via Bernina 21, Centro, Camparada"
2,IMMOBILBIELLA,,si,1 in garagebox,unknown,Piano terra,Caminetto Impianto di allarme Porta blindata...,Ottimo Ristrutturato,"Appartamento viale Roma 12, Masarone, Biella"
3,Immobiliare Aguiari,,si,1 in garagebox,si,1°,Porta blindata Balcone Impianto tv centraliz...,Ottimo Ristrutturato,"Quadrilocale Strada della Nera 21, Piazzo, Van..."
4,REMAX Unit srl,,None,"1 in garagebox, 1",unknown,Piano terra,Caminetto Fibra ottica Armadio a muro Impia...,Buono Abitabile,"Quadrilocale Lorazzo Inferiore 10, Andorno Cac..."
...,...,...,...,...,...,...,...,...,...
9986,Affiliato Tecnocasa: TECNOPONTE SRL,,si,,si,5° piano,Balcone Arredato Cantina,Buono Abitabile,"Trilocale viale Derna 40, Centro, Edolo"
9987,Valcasa srl,,si,,unknown,1°,Esposizione esterna Balcone Cantina Giardin...,Buono Abitabile,"Terratetto unifamiliare 110 m², buono stato, V..."
9988,Affiliato Tecnocasa: TECNOTIRANO SRL,,si,,unknown,,Balcone Parzialmente Arredato Cantina Giard...,Da ristrutturare,"Terratetto plurifamiliare via Gilera, Centro, ..."
9989,REMAX Next,,si,1 in garagebox,si,1 piano: Piano terra,Cancello elettrico Fibra ottica Porta blinda...,Ottimo Ristrutturato,"Bilocale via Galet 1, Valdisotto"


### Step 17 and 18: Fill missing
In this step we will fill missing values with N/A of string columns and with 0 for numeric columns.

In [11]:
fill_df = remove_symbols_df.copy()
string_columns = [
            "data_agency",
            "data_arredo",
            "data_balcony",
            "data_box",
            "data_coords",
            "data_elevator",
            "data_floor",
            "data_status",
            "data_room_info_feature_list",
            "data_title"
          ]
numeric_columns = [
            "data_baths",
            "data_rooms"
          ]
fill_df[string_columns] = fill_df[string_columns].fillna(value="N/A")
fill_df[numeric_columns] = fill_df[numeric_columns].fillna(value=0)
fill_df

,data_box,data_size,data_baths,data_floor,data_price,data_rooms,data_title,data_agency,data_arredo,data_coords,data_status,data_balcony,data_elevator,data_publish_date,data_room_info_feature_list,data_publish_date_year,data_publish_date_quarter,data_publish_date_month
0,1 in garagebox,68.0,1,N/A,115000.0,2,"Bilocale via Comotto, Villafranca di Verona",Immobiliare Duomo,,"45.3839,10.9187",Buono Abitabile,N/A,unknown,2023-02-22 18:45:20,,2023,0,1
1,1 in garagebox,80.0,2,3° piano,155000.0,3,"Trilocale via Bernina 21, Centro, Camparada",TEMPOCASA CASATENOVO,,"45.6549,9.32357",Buono Abitabile,N/A,unknown,2022-09-21 00:00:00,Cancello elettrico Mansarda Porta blindata ...,2022,2,8
2,1 in garagebox,348.0,3,Piano terra,400000.0,5,"Appartamento viale Roma 12, Masarone, Biella",IMMOBILBIELLA,,"45.5563,8.0653",Ottimo Ristrutturato,si,unknown,2022-04-21 00:00:00,Caminetto Impianto di allarme Porta blindata...,2022,1,3
3,1 in garagebox,90.0,1,1°,65000.0,4,"Quadrilocale Strada della Nera 21, Piazzo, Van...",Immobiliare Aguiari,,"45.5709,8.0406",Ottimo Ristrutturato,si,si,2022-02-03 00:00:00,Porta blindata Balcone Impianto tv centraliz...,2022,0,1
4,"1 in garagebox, 1",98.0,1,Piano terra,75000.0,4,"Quadrilocale Lorazzo Inferiore 10, Andorno Cac...",REMAX Unit srl,,"45.6016,8.0525",Buono Abitabile,N/A,unknown,2022-06-01 00:00:00,Caminetto Fibra ottica Armadio a muro Impia...,2022,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,,99.0,1,5° piano,60000.0,3,"Trilocale viale Derna 40, Centro, Edolo",Affiliato Tecnocasa: TECNOPONTE SRL,,"46.1786,10.3291",Buono Abitabile,si,si,2022-03-23 00:00:00,Balcone Arredato Cantina,2022,0,2
9987,,110.0,2,1°,92000.0,4,"Terratetto unifamiliare 110 m², buono stato, V...",Valcasa srl,,"46.2507,10.2361",Buono Abitabile,si,unknown,2021-12-13 00:00:00,Esposizione esterna Balcone Cantina Giardin...,2021,3,11
9988,,250.0,2,,35000.0,3,"Terratetto plurifamiliare via Gilera, Centro, ...",Affiliato Tecnocasa: TECNOTIRANO SRL,,"46.2123,10.1733",Da ristrutturare,si,unknown,2022-05-19 00:00:00,Balcone Parzialmente Arredato Cantina Giard...,2022,1,4
9989,1 in garagebox,45.0,1,1 piano: Piano terra,110000.0,2,"Bilocale via Galet 1, Valdisotto",REMAX Next,,"46.4624,10.3415",Ottimo Ristrutturato,si,si,2022-07-08 00:00:00,Cancello elettrico Fibra ottica Porta blinda...,2022,2,6


### Step 19: Group Agency
To apply custom transformations we can use Data Wrangler functionalities.

*Use Pyspark, Pandas, or Pyspark (SQL) to define custom transformations.*

https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html#data-wrangler-transform-custom

In the following cell we create a custom function that uses pandas dataseries to transform the data_agency column.

<img src="../images/19_group_agency.png" alt="19. Group Agency with Data Wrangler Python User-Defined Function" width="auto"/>

We decided to group agencies in three different categories: franchising, independent agencies and private.


In [12]:
group_agency_df = fill_df.copy()
def group_agency_func(series: pd.Series) -> pd.Series:
    
    series = series.str.lower()
    
    franchising_list = [
        'abytare','tecnocasa','fondocasa','neovit','tecnorete','engel','frimm','gabetti','grimaldi','habitat','professionecasa',
        'remax','retecasa','tecnoaffari','tempocasa','vendocasa','unicredit','soloaffitti','toscano','studiocasa','iconacasa','coldwell',
        'primacasa','unica','retecasa','mediocasa','sarpi','imi','progettocasa']
    
    series[series.str.contains('|'.join(franchising_list))] = 'franchising'
    series[~series.str.contains('|'.join(['privato','franchising']))] = 'agency'
    return series

group_agency_df["group_agency"] = group_agency_func(group_agency_df["data_agency"])
group_agency_df[["data_agency","group_agency"]]

,data_agency,group_agency
0,Immobiliare Duomo,agency
1,TEMPOCASA CASATENOVO,franchising
2,IMMOBILBIELLA,agency
3,Immobiliare Aguiari,agency
4,REMAX Unit srl,franchising
...,...,...
9986,Affiliato Tecnocasa: TECNOPONTE SRL,franchising
9987,Valcasa srl,agency
9988,Affiliato Tecnocasa: TECNOTIRANO SRL,franchising
9989,REMAX Next,franchising


### Step 19 and 20: Box Num and Box Type

We are going to use custom trasform again with data_box column, this time we will use a python function that take as input a single value.
In this way we will split data_box in two new columns box_num indicating the number of spaces availables and box_type indicating if the box is inside, outside or shared.

In [13]:
import re
box_df = group_agency_df.copy()

def box_num_func(value):
    split_l = re.split(r"(\d+)", value.strip())
    if len(split_l) > 2:
        return float(split_l[1].replace(",",""))
    return 0.0

def box_type_func(value):
    split_l = re.split(r"(\d+)", value.strip())
    if len(split_l) > 2:
        box_type = split_l[2].replace(",","")
    else:
        box_type = "N/A"

    if "esterno" in box_type:
        box_type  = "esterno"
    elif "privato" in box_type:
        box_type  = "privato"
    elif "comune" in box_type:
        box_type  = "comune"
    elif "box" in box_type:
        box_type  = "box"
    else:
        box_type = "N/A"
    return box_type

box_df["box_num"] = box_df["data_box"].apply(lambda x: box_num_func(x))
box_df["box_type"] = box_df["data_box"].apply(lambda x: box_type_func(x))

box_df[["data_box","box_num","box_type"]]

,data_box,box_num,box_type
0,1 in garagebox,1.0,box
1,1 in garagebox,1.0,box
2,1 in garagebox,1.0,box
3,1 in garagebox,1.0,box
4,"1 in garagebox, 1",1.0,box
...,...,...,...
9986,,0.0,N/A
9987,,0.0,N/A
9988,,0.0,N/A
9989,1 in garagebox,1.0,box


### Step 22: ConvertFloor
In this step we are going to convert data_floor using custom function.

In [14]:
floor_df = box_df.copy()
def floor_num_func(value):
    piano = value
    try:
        return str(float(piano))
    except:
        pass
    try:
        normalized_piano = str(piano).lower()
        if "seminterrato" in normalized_piano or "semi-interrato" in normalized_piano or "interrato" in normalized_piano:
            return '-1'
        if 'terra' in normalized_piano:
            return '0'
        if 'rialzato' in normalized_piano or 'ammezzato' in normalized_piano:
            return '0.5'
        if 'su più livelli' in normalized_piano:
            return "N/A"
        if 'oltre il decimo' in normalized_piano:
            return '10'
        if '°' in normalized_piano:
            v = normalized_piano.split('°')[0]
            r = re.findall('\d+', v)
            if len(r) > 0:
                return str(r[-1])
            return "N/A"
        for el in ["sopra il", "°", "ultimo di", "ª", "+", " piano", ":", "planta"]:
            if el in normalized_piano:
                v = normalized_piano.split('°')[0]
                r = re.findall('\d+', v)
            if len(r) > 0:
                return str(r[-1])
            return "N/A"
        if "di" in normalized_piano:
            if el in normalized_piano:
                v = normalized_piano.split('°')[0]
                r = re.findall('\d+', v)
            if len(r) > 0:
                return str(r[-1])
            return "N/A"
    except:
        return "N/A"
    return "N/A"

floor_df["floor_num"] = floor_df["data_floor"].apply(lambda x: floor_num_func(x))
floor_df[["data_floor", "floor_num"]]

,data_floor,floor_num
0,N/A,N/A
1,3° piano,3
2,Piano terra,0
3,1°,1
4,Piano terra,0
...,...,...
9986,5° piano,5
9987,1°,1
9988,,N/A
9989,1 piano: Piano terra,0


### Step 23: GroupTitle
In this step we are going to group titles with specific categories.

In [15]:
group_title_df = floor_df.copy()

def group_title_func(value):
    title = value.lower()
    if "appartamento" in title:
        return "appartamento"
    elif "attico" in title:
        return "attico"
    elif "casale" in title:
        return "casale"
    elif "cascina" in title:
        return "cascina"
    elif "loft" in title:
        return "loft"
    elif "mansarda" in title:
        return "mansarda"
    elif "bilocale" in title:
        return "bilocale"
    elif "monolocale" in title:
        return "monolocale"
    elif "quadrilocale" in title:
        return "quadrilocale"
    elif "rustico" in title:
        return "rustico"
    elif "terratetto" in title:
        return "terratetto"
    elif "trilocale" in title:
        return "trilocale"
    elif "schiera" in title:
        return "villa a schiera"
    elif "bifamiliare" in title:
        return "villa bifamiliare"
    elif "plurifamiliare" in title:
        return "villa plurifamiliare"
    elif "unifamiliare" in title:
        return "villa unifamiliare"
    else:
        return None
    
group_title_df["dwelling_type"] = group_title_df["data_title"].apply(lambda x: group_title_func(x))
group_title_df[["data_title", "dwelling_type"]]

,data_title,dwelling_type
0,"Bilocale via Comotto, Villafranca di Verona",bilocale
1,"Trilocale via Bernina 21, Centro, Camparada",trilocale
2,"Appartamento viale Roma 12, Masarone, Biella",appartamento
3,"Quadrilocale Strada della Nera 21, Piazzo, Van...",quadrilocale
4,"Quadrilocale Lorazzo Inferiore 10, Andorno Cac...",quadrilocale
...,...,...
9986,"Trilocale viale Derna 40, Centro, Edolo",trilocale
9987,"Terratetto unifamiliare 110 m², buono stato, V...",terratetto
9988,"Terratetto plurifamiliare via Gilera, Centro, ...",terratetto
9989,"Bilocale via Galet 1, Valdisotto",bilocale


### Step 24 and 25: Split data_coords and flatten data_coords
In this step we are going to split data_coords columns in two new columns representing latitude and longitude.

In [16]:
coords_df = group_title_df.copy()

def get_coords(value, index):
    try:
        return float(value.split(",")[index])
    except:
        return np.nan
    
coords_df["data_coords_0"] = coords_df['data_coords'].apply(lambda x:  get_coords(x,0))
coords_df["data_coords_1"] = coords_df['data_coords'].apply(lambda x: get_coords(x,1))
print(coords_df.shape)
coords_df = coords_df[~coords_df["data_coords_0"].isna()]
coords_df = coords_df[~coords_df["data_coords_1"].isna()]
coords_df[["data_coords", "data_coords_0", "data_coords_1"]]


(9363, 25)


,data_coords,data_coords_0,data_coords_1
0,"45.3839,10.9187",45.3839,10.91870
1,"45.6549,9.32357",45.6549,9.32357
2,"45.5563,8.0653",45.5563,8.06530
3,"45.5709,8.0406",45.5709,8.04060
4,"45.6016,8.0525",45.6016,8.05250
...,...,...,...
9986,"46.1786,10.3291",46.1786,10.32910
9987,"46.2507,10.2361",46.2507,10.23610
9988,"46.2123,10.1733",46.2123,10.17330
9989,"46.4624,10.3415",46.4624,10.34150


### Step 26 and 27: Drop missing dwelling_type and drop columns
In this step we are going to drop missing values for dwelling_type columns.

In [17]:
dwelling_df = coords_df.copy()
print(dwelling_df.shape)

dwelling_df = dwelling_df[dwelling_df["dwelling_type"].notna()]
print(dwelling_df.shape)

columns_to_drop = [
            "data_title",
            "data_publish_date",
            "data_agency",
            "data_box",
            "data_floor",
            "data_coords"
          ]
dwelling_df = dwelling_df.drop(columns=columns_to_drop, axis=1)
dwelling_df

(9361, 25)
(9249, 25)


,data_size,data_baths,data_price,data_rooms,data_arredo,data_status,data_balcony,data_elevator,data_room_info_feature_list,data_publish_date_year,data_publish_date_quarter,data_publish_date_month,group_agency,box_num,box_type,floor_num,dwelling_type,data_coords_0,data_coords_1
0,68.0,1,115000.0,2,,Buono Abitabile,N/A,unknown,,2023,0,1,agency,1.0,box,N/A,bilocale,45.3839,10.91870
1,80.0,2,155000.0,3,,Buono Abitabile,N/A,unknown,Cancello elettrico Mansarda Porta blindata ...,2022,2,8,franchising,1.0,box,3,trilocale,45.6549,9.32357
2,348.0,3,400000.0,5,,Ottimo Ristrutturato,si,unknown,Caminetto Impianto di allarme Porta blindata...,2022,1,3,agency,1.0,box,0,appartamento,45.5563,8.06530
3,90.0,1,65000.0,4,,Ottimo Ristrutturato,si,si,Porta blindata Balcone Impianto tv centraliz...,2022,0,1,agency,1.0,box,1,quadrilocale,45.5709,8.04060
4,98.0,1,75000.0,4,,Buono Abitabile,N/A,unknown,Caminetto Fibra ottica Armadio a muro Impia...,2022,1,5,franchising,1.0,box,0,quadrilocale,45.6016,8.05250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,99.0,1,60000.0,3,,Buono Abitabile,si,si,Balcone Arredato Cantina,2022,0,2,franchising,0.0,N/A,5,trilocale,46.1786,10.32910
9987,110.0,2,92000.0,4,,Buono Abitabile,si,unknown,Esposizione esterna Balcone Cantina Giardin...,2021,3,11,agency,0.0,N/A,1,terratetto,46.2507,10.23610
9988,250.0,2,35000.0,3,,Da ristrutturare,si,unknown,Balcone Parzialmente Arredato Cantina Giard...,2022,1,4,franchising,0.0,N/A,N/A,terratetto,46.2123,10.17330
9989,45.0,1,110000.0,2,,Ottimo Ristrutturato,si,si,Cancello elettrico Fibra ottica Porta blinda...,2022,2,6,franchising,1.0,box,0,bilocale,46.4624,10.34150


### Step 28: One-hot encoding

In this step we are going to apply One-hot encoding to our categorical columns.

https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html#data-wrangler-transform-cat-encode

*Select One-hot encode for Transform to use one-hot encoding.*

*Configure this transform using the following:*
- *Drop last category: If True, the last category does not have a corresponding index in the one-hot encoding. When missing values are possible, a missing category is always the last one and setting this to True means that a missing value results in an all zero vector.*
- *Invalid handing strategy: Select a method to handle invalid or missing values.*
    - *Choose Skip if you want to omit the rows with missing values.*
    - *Choose Keep to retain missing values as the last category.*
    - *Choose Error if you want Data Wrangler to throw an error if missing values are encountered in the Input column.*
- *Is input ordinal encoded: Select this option if the input vector contains ordinal encoded data. This option requires that input data contain non-negative integers. If True, input i is encoded as a vector with a non-zero in the ith location.*

<img src="../images/28_one_hot.png" alt="28. One-hot encoding with Data Wrangler" width="auto"/>

We are going to use pandas.get_dummies function to apply one-hot encoding.

In [18]:
one_hot_df = dwelling_df.copy()

one_hot_columns = [
            "data_arredo",
            "data_balcony",
            "data_elevator",
            "data_status",
            "dwelling_type",
            "box_type",
            "floor_num",
            "group_agency"
          ]

one_hot_df = pd.get_dummies(one_hot_df, columns=one_hot_columns, drop_first=True)
one_hot_df

,data_size,data_baths,data_price,data_rooms,data_room_info_feature_list,data_publish_date_year,data_publish_date_quarter,data_publish_date_month,box_num,data_coords_0,...,floor_num_3,floor_num_4,floor_num_5,floor_num_6,floor_num_7,floor_num_8,floor_num_9,floor_num_N/A,group_agency_franchising,group_agency_privato
0,68.0,1,115000.0,2,,2023,0,1,1.0,45.3839,...,0,0,0,0,0,0,0,1,0,0
1,80.0,2,155000.0,3,Cancello elettrico Mansarda Porta blindata ...,2022,2,8,1.0,45.6549,...,1,0,0,0,0,0,0,0,1,0
2,348.0,3,400000.0,5,Caminetto Impianto di allarme Porta blindata...,2022,1,3,1.0,45.5563,...,0,0,0,0,0,0,0,0,0,0
3,90.0,1,65000.0,4,Porta blindata Balcone Impianto tv centraliz...,2022,0,1,1.0,45.5709,...,0,0,0,0,0,0,0,0,0,0
4,98.0,1,75000.0,4,Caminetto Fibra ottica Armadio a muro Impia...,2022,1,5,1.0,45.6016,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,99.0,1,60000.0,3,Balcone Arredato Cantina,2022,0,2,0.0,46.1786,...,0,0,1,0,0,0,0,0,1,0
9987,110.0,2,92000.0,4,Esposizione esterna Balcone Cantina Giardin...,2021,3,11,0.0,46.2507,...,0,0,0,0,0,0,0,0,0,0
9988,250.0,2,35000.0,3,Balcone Parzialmente Arredato Cantina Giard...,2022,1,4,0.0,46.2123,...,0,0,0,0,0,0,0,1,1,0
9989,45.0,1,110000.0,2,Cancello elettrico Fibra ottica Porta blinda...,2022,2,6,1.0,46.4624,...,0,0,0,0,0,0,0,0,1,0


### Step 29 and  30: Vectorize data_room_info_feature_list and drop it
In this step we are going to vectorize data_room_info_feature_list. Meaning we will mapping words or phrases from a vocabulary to vectors of real numbers.

https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html#data-wrangler-transform-featurize-text

<img src="../images/29_vectorize.png" alt="29. Vectorize with Data Wrangler" width="auto"/>

We will use scikit-learn TfidfVectorizer to obtain similar results.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize_df = one_hot_df.copy()

tfidf_vectorizer = TfidfVectorizer()

doc_vec  = tfidf_vectorizer.fit_transform(vectorize_df["data_room_info_feature_list"])
tf_idf_df = pd.DataFrame(doc_vec.toarray().transpose(), index=tfidf_vectorizer.get_feature_names()).transpose()

new_columns = []
for c in tf_idf_df.columns:
        new_columns.append( f"data_room_info_feature_list_{c}")
tf_idf_df.columns = new_columns

vectorize_df = pd.merge(vectorize_df, tf_idf_df, how="inner", left_index=True, right_index=True)
vectorize_df[["data_room_info_feature_list", "data_room_info_feature_list_cancello", "data_room_info_feature_list_porta"]]

,data_room_info_feature_list,data_room_info_feature_list_cancello,data_room_info_feature_list_porta
0,,0.000000,0.000000
1,Cancello elettrico Mansarda Porta blindata ...,0.248624,0.204971
2,Caminetto Impianto di allarme Porta blindata...,0.000000,0.205270
3,Porta blindata Balcone Impianto tv centraliz...,0.000000,0.219477
4,Caminetto Fibra ottica Armadio a muro Impia...,0.000000,0.000000
...,...,...,...
9243,,0.000000,0.245090
9244,VideoCitofono Porta blindata 3 Balconi Port...,0.000000,0.000000
9245,Cancello elettrico Fibra ottica VideoCitofon...,0.000000,0.000000
9247,Porta blindata Balcone Impianto tv centraliz...,0.191186,0.157617


In [20]:
vectorize_df = vectorize_df.drop("data_room_info_feature_list", axis=1)
vectorize_df

,data_size,data_baths,data_price,data_rooms,data_publish_date_year,data_publish_date_quarter,data_publish_date_month,box_num,data_coords_0,data_coords_1,...,data_room_info_feature_list_satellitare,data_room_info_feature_list_singolo,data_room_info_feature_list_solo,data_room_info_feature_list_sud,data_room_info_feature_list_taverna,data_room_info_feature_list_terrazza,data_room_info_feature_list_triplo,data_room_info_feature_list_tv,data_room_info_feature_list_vetro,data_room_info_feature_list_videocitofono
0,68.0,1,115000.0,2,2023,0,1,1.0,45.3839,10.91870,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,80.0,2,155000.0,3,2022,2,8,1.0,45.6549,9.32357,...,0.0,0.0,0.0,0.0,0.0,0.234315,0.0,0.185476,0.170102,0.0
2,348.0,3,400000.0,5,2022,1,3,1.0,45.5563,8.06530,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.185747,0.170350,0.0
3,90.0,1,65000.0,4,2022,0,1,1.0,45.5709,8.04060,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.198602,0.182140,0.0
4,98.0,1,75000.0,4,2022,1,5,1.0,45.6016,8.05250,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.176903,0.162239,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9243,100.0,1,180000.0,5,2023,1,3,1.0,43.8501,10.49230,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.221780,0.203397,0.0
9244,140.0,3,299000.0,4,2022,2,6,0.0,45.3945,9.21506,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
9245,500.0,3,5080000.0,5,2022,1,4,0.0,45.4601,9.17710,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.244020,0.0
9247,50.0,1,45000.0,2,2022,3,11,0.0,45.0926,7.69690,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.130804,0.0


### Step 31 and 32: Drop duplicates and move data_price to first column

Here we are going to drop duplicates and move data_price as first column because Sagemaker Algorithm need it when storing data as .csv files.

In [21]:
ready_df = vectorize_df.copy()
print(ready_df.shape)

ready_df.drop_duplicates()
ready_df.insert(0, "data_price", ready_df.pop("data_price"))
ready_df

(8562, 114)


,data_price,data_size,data_baths,data_rooms,data_publish_date_year,data_publish_date_quarter,data_publish_date_month,box_num,data_coords_0,data_coords_1,...,data_room_info_feature_list_satellitare,data_room_info_feature_list_singolo,data_room_info_feature_list_solo,data_room_info_feature_list_sud,data_room_info_feature_list_taverna,data_room_info_feature_list_terrazza,data_room_info_feature_list_triplo,data_room_info_feature_list_tv,data_room_info_feature_list_vetro,data_room_info_feature_list_videocitofono
0,115000.0,68.0,1,2,2023,0,1,1.0,45.3839,10.91870,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,155000.0,80.0,2,3,2022,2,8,1.0,45.6549,9.32357,...,0.0,0.0,0.0,0.0,0.0,0.234315,0.0,0.185476,0.170102,0.0
2,400000.0,348.0,3,5,2022,1,3,1.0,45.5563,8.06530,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.185747,0.170350,0.0
3,65000.0,90.0,1,4,2022,0,1,1.0,45.5709,8.04060,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.198602,0.182140,0.0
4,75000.0,98.0,1,4,2022,1,5,1.0,45.6016,8.05250,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.176903,0.162239,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9243,180000.0,100.0,1,5,2023,1,3,1.0,43.8501,10.49230,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.221780,0.203397,0.0
9244,299000.0,140.0,3,4,2022,2,6,0.0,45.3945,9.21506,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
9245,5080000.0,500.0,3,5,2022,1,4,0.0,45.4601,9.17710,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.244020,0.0
9247,45000.0,50.0,1,2,2022,3,11,0.0,45.0926,7.69690,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.130804,0.0


### Step 33: Join with cluster means

As described in the previous notebook we can use coordinates data to cluster results.
In this case, we are going to merge our ready_df with the cluster_means we identified.

As an example we are going to load cluster_mean.csv containing information on cluster means and join it with our current dataframe.

In [22]:
cluster_s3_uri = "s3://ds-houseprices/cluster_mean.csv"
cluster_df = pd.read_csv(cluster_s3_uri)

final_df = pd.merge(ready_df, cluster_df, how="inner", left_on=["data_coords_0", "data_coords_1"], right_on=["latitude", "longitude"])
final_df = final_df.drop(columns=["data_coords_0", "data_coords_1", "latitude", "longitude"], axis=1)
final_df

,data_price,data_size,data_baths,data_rooms,data_publish_date_year,data_publish_date_quarter,data_publish_date_month,box_num,data_arredo_Arredato,data_arredo_Parzialmente Arredato,...,data_room_info_feature_list_singolo,data_room_info_feature_list_solo,data_room_info_feature_list_sud,data_room_info_feature_list_taverna,data_room_info_feature_list_terrazza,data_room_info_feature_list_triplo,data_room_info_feature_list_tv,data_room_info_feature_list_vetro,data_room_info_feature_list_videocitofono,cluster_mean
0,115000.0,68.0,1,2,2023,0,1,1.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1562.203278
1,155000.0,80.0,2,3,2022,2,8,1.0,0,0,...,0.0,0.0,0.0,0.0,0.234315,0.0,0.185476,0.170102,0.0,1830.723879
2,400000.0,348.0,3,5,2022,1,3,1.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.185747,0.170350,0.0,741.062962
3,9900.0,14.0,1,1,2021,3,11,0.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.184614,0.0,741.062962
4,65000.0,90.0,1,4,2022,0,1,1.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.198602,0.182140,0.0,828.237424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8481,170000.0,600.0,2,5,2022,2,8,1.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.181917,0.0,1470.990727
8482,180000.0,100.0,1,5,2023,1,3,1.0,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.221780,0.203397,0.0,2267.658416
8483,299000.0,140.0,3,4,2022,2,6,0.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,2440.084957
8484,45000.0,50.0,1,2,2022,3,11,0.0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.130804,0.0,942.606956


## Conclusion
To conclude this cleaning step we are going to split our dataframe in three different datasets with the *randomized split* function of the Data Wrangler.

https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html#data-wrangler-transform-split-data

<img src="../images/randomized_split.png" alt="Randomized split with Data Wrangler" width="auto"/>

In this case we are going to use 90% of our datafram as training set, while 5% as validation set and 5% test set.

This step needs to be connected with three different destinations that are connected with an s3 bucket.

Additionally, we are going to remove the header from the saved .csv since Sagemaker built-in algorithms want this format for training:

https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html

In [23]:
# This are the s3 object names for the folder where our data will be saved
numeric_prefix = f"ETL/ETL_Numeric"
numeric_categorical_prefix = f"ETL/ETL_Numeric_Categorical"

train_data_key = f"{numeric_prefix}/Train/noheader"
validation_data_key = f"{numeric_prefix}/Validation/noheader"
test_data_key = f"{numeric_prefix}/Test/noheader"

## Further steps with mixed numerical and categorical datasets
As further step we created a similar Data Wrangler flow which results are mixed numerical and categorical data.

We have decided to create also this kind of dataset to test various algorithm that Sagemaker provides like:
- AutoGluon https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html
- CatBoost https://docs.aws.amazon.com/sagemaker/latest/dg/catboost.html

Now let's continue on with the next step, where we are going to train various models with our datasets.

[Go to Modeling](modeling.ipynb)